In [1]:
import pandas as pd
import time
from sklearn.model_selection import StratifiedGroupKFold, StratifiedKFold
from sklearn.linear_model import LogisticRegression

import numpy as np
from matplotlib import pyplot as plt
import warnings
from skopt import BayesSearchCV, space
from sklearn.metrics import average_precision_score
# use feature importance for feature selection
from numpy import sort
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectFromModel    
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
warnings.filterwarnings("ignore")
from sklearn.datasets import load_iris
from sklearn.model_selection import GridSearchCV, KFold, cross_val_score, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from mlxtend.classifier import EnsembleVoteClassifier
from xgboost import XGBClassifier
#from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from scipy.stats import uniform, randint
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.ensemble import VotingClassifier
from sklearn.calibration import CalibratedClassifierCV
### Parameters
from sklearn.linear_model import Lasso
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix
from autogluon.core.metrics import make_scorer
from autogluon.tabular import TabularPredictor
from autogluon.common import space
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor
from sklearn.model_selection import KFold
#import ts_fc_functions as ff
import import_ipynb
from darts import TimeSeries
from darts.models import NBEATSModel, XGBModel, TiDEModel#, LightGBMModel
from darts.models import NHiTSModel,LightGBMModel,RNNModel
from darts.dataprocessing.transformers import Scaler, MissingValuesFiller
from darts.metrics import mape, r2_score, mae, rmse
import torch

import optuna
from optuna.integration import PyTorchLightningPruningCallback
from pytorch_lightning.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split


In [ ]:
def process_use_case_3_data(data_use_case_3, start_date, end_date):
    df_sorted = data_use_case_3.sort_values(by='performedDateTime').reset_index(drop=True)
    df_no_duplicates = df_sorted.drop_duplicates(subset=['performedDateTime', 'subject_reference', 'encounter_reference']).reset_index(drop=True)

    # Generate a date range with unique dates
    date_range = pd.date_range(start=start_date, end=end_date, freq="D")

    # Duplicate each date to have two entries per day
    date_range = date_range.repeat(2)
    df_no_duplicates["date"] = pd.to_datetime(df_no_duplicates["date"])


    ctscans = pd.DataFrame(date_range, columns=["date"])
    ctscans["time_of_day"] = ctscans.groupby("date").cumcount().map({0: "a_morning", 1: "b_evening"})

    count_df = df_no_duplicates.groupby(["date", "time_of_day"]).size().reset_index(name="count_scans")
    ctscans = pd.merge(ctscans, count_df, on=["date", "time_of_day"], how="left")
    ctscans["count_scans"] = ctscans["count_scans"].fillna(0).astype(int)
    ctscans = ctscans.drop(index=0)

    # Reset the index if desired, to avoid a missing index number
    ctscans = ctscans.reset_index(drop=True)

    # Create a new column 'shift_count' initialized to NaN
    ctscans["shift_count"] = np.nan

    # Iterate over the rows and sum count_scans for each consecutive "b_evening" and "a_morning" pair
    for i in range(len(ctscans) - 1):
        if ctscans.loc[i, "time_of_day"] == "b_evening" and ctscans.loc[i + 1, "time_of_day"] == "a_morning":
            # Sum count_scans for the consecutive "b_evening" and "a_morning"
            shift_sum = ctscans.loc[i, "count_scans"] + ctscans.loc[i + 1, "count_scans"]
            
            # Assign the result to the first row of the pair ("b_evening" row)
            ctscans.loc[i, "shift_count"] = shift_sum

    # Optionally, forward-fill or set NaN values to 0 in 'shift_count'
    ctscans["shift_count"] = ctscans["shift_count"].fillna(0).astype(int)
    ctscans = ctscans[ctscans["time_of_day"] != "a_morning"].reset_index(drop=True)
    ctscans["day_of_week"] = ctscans["date"].dt.day_name()
    last_friday = ctscans[ctscans["day_of_week"] == "Friday"]["date"].max()

    # Step 2: Initialize ts_id counter and end_date for the first 4-week period
    current_id = 1
    end_date = last_friday

    # Step 3: Create an empty 'ts_id' column
    ctscans["ts_id"] = 0

    # Step 4: Assign ts_id to each 4-week (28-day) period, going backwards from the last Friday
    while end_date >= ctscans["date"].min():
        start_date = end_date - pd.Timedelta(weeks=4)

        # Assign the current ts_id to dates within the 4-week range
        ctscans.loc[(ctscans["date"] > start_date) & (ctscans["date"] <= end_date), "ts_id"] = current_id

        # Move to the next 4-week period back in time
        end_date = start_date
        current_id += 1
    ctscans
    return ctscans

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import pandas as pd
import numpy as np

def process_use_case_3_data_with_scaling(data_use_case_3,start_date,end_date):
    df_sorted = data_use_case_3.sort_values(by='performedDateTime').reset_index(drop=True)
    df_no_duplicates = df_sorted.drop_duplicates(subset=['performedDateTime', 'subject_reference', 'encounter_reference']).reset_index(drop=True)

    # Generate a date range with unique dates
    date_range = pd.date_range(start=start_date, end=end_date, freq="D")

    # Duplicate each date to have two entries per day
    date_range = date_range.repeat(2)
    df_no_duplicates["date"] = pd.to_datetime(df_no_duplicates["date"])

    ctscans = pd.DataFrame(date_range, columns=["date"])
    ctscans["time_of_day"] = ctscans.groupby("date").cumcount().map({0: "a_morning", 1: "b_evening"})

    count_df = df_no_duplicates.groupby(["date", "time_of_day"]).size().reset_index(name="count_scans")
    ctscans = pd.merge(ctscans, count_df, on=["date", "time_of_day"], how="left")
    ctscans["count_scans"] = ctscans["count_scans"].fillna(0).astype(int)
    ctscans = ctscans.drop(index=0)

    # Reset the index if desired, to avoid a missing index number
    ctscans = ctscans.reset_index(drop=True)

    # Create a new column 'shift_count' initialized to NaN
    ctscans["shift_count"] = np.nan

    # Iterate over the rows and sum count_scans for each consecutive "b_evening" and "a_morning" pair
    for i in range(len(ctscans) - 1):
        if ctscans.loc[i, "time_of_day"] == "b_evening" and ctscans.loc[i + 1, "time_of_day"] == "a_morning":
            # Sum count_scans for the consecutive "b_evening" and "a_morning"
            shift_sum = ctscans.loc[i, "count_scans"] + ctscans.loc[i + 1, "count_scans"]
            
            # Assign the result to the first row of the pair ("b_evening" row)
            ctscans.loc[i, "shift_count"] = shift_sum

    # Optionally, forward-fill or set NaN values to 0 in 'shift_count'
    ctscans["shift_count"] = ctscans["shift_count"].fillna(0).astype(int)
    ctscans = ctscans[ctscans["time_of_day"] != "a_morning"].reset_index(drop=True)
    ctscans["day_of_week"] = ctscans["date"].dt.day_name()
    last_friday = ctscans[ctscans["day_of_week"] == "Friday"]["date"].max()

    # Step 2: Initialize ts_id counter and end_date for the first 4-week period
    current_id = 1
    end_date = last_friday

    # Step 3: Create an empty 'ts_id' column
    ctscans["ts_id"] = 0

    # Step 4: Assign ts_id to each 4-week (28-day) period, going backwards from the last Friday
    while end_date >= ctscans["date"].min():
        start_date = end_date - pd.Timedelta(weeks=4)

        # Assign the current ts_id to dates within the 4-week range
        ctscans.loc[(ctscans["date"] > start_date) & (ctscans["date"] <= end_date), "ts_id"] = current_id

        # Move to the next 4-week period back in time
        end_date = start_date
        current_id += 1

    # Min-Max scaling for 'count_scans' and 'shift_count'
    scaler = MinMaxScaler()
    scaled_columns = ["shift_count"]

    # Apply Min-Max Scaling
    ctscans[scaled_columns] = scaler.fit_transform(ctscans[scaled_columns])

    return ctscans, scaler

In [ ]:
def create_rolling_windows_with_weekday_prediction(data, target_col, lookback, horizon, prediction_days):
    """
    Creates rolling windows for a time series while preserving dates for each window.
    Args:
        data (DataFrame): DataFrame containing the time series.
        target_col (str): Column name of the target variable.
        lookback (int): Number of steps to look back.
        horizon (int): Number of steps to forecast.
        prediction_days (list): List of valid day names for the prediction window (e.g., ['Monday', ..., 'Friday']).
    Returns:
        X (list): List of dictionaries containing input sequences and their corresponding dates.
        y (list): List of dictionaries containing target sequences and their corresponding dates.
        dates (list): Start dates of each rolling window.
    """
    X, y, dates = [], [], []

    for i in range(len(data) - lookback - horizon + 1):
        # Define the input (lookback) window
        window = data.iloc[i:i + lookback]
        input_values = window[target_col].values
        input_dates = window['date'].values
        
        # Define the prediction (horizon) window
        prediction_window = data.iloc[i + lookback:i + lookback + horizon]
        target_values = prediction_window[prediction_window['day_of_week'].isin(prediction_days)][target_col].values
        target_dates = prediction_window[prediction_window['day_of_week'].isin(prediction_days)]['date'].values

        # Ensure the prediction window matches the expected length
        if len(target_values) == len(prediction_days):
            X.append({"values": input_values, "dates": input_dates})
            y.append({"values": target_values, "dates": target_dates})
            dates.append(data.iloc[i + lookback]['date'])

    def convert_to_dataframe_with_id(data, id_name):
        """
        Converts a list of dictionaries with 'values' and 'dates' into a DataFrame, 
        assigning a unique id to each sequence.
        Args:
            data (list): List of dictionaries with keys 'values' and 'dates'.
            id_name (str): Column name for the unique identifier.
        Returns:
            DataFrame: Combined DataFrame of all sequences with dates, values, and ids.
        """
        df_list = []
        for idx, entry in enumerate(data, start=1):
            temp_df = pd.DataFrame({
                "date": entry["dates"],
                "value": entry["values"],
                id_name: idx  # Assign a unique ID for each sequence
            })
            df_list.append(temp_df)
        return pd.concat(df_list, ignore_index=True)

    # Example X and y conversion with IDs

    X_df = convert_to_dataframe_with_id(X, id_name="X_id")
    y_df = convert_to_dataframe_with_id(y, id_name="y_id")
    def merge_and_order_dataframes(X_df, y_df, X_id_col="X_id", y_id_col="y_id"):
        """
        Merges X_df and y_df, ordering by id and date, and distinguishing X and y records.
        Args:
            X_df (DataFrame): DataFrame for X with date, value, and ID.
            y_df (DataFrame): DataFrame for y with date, value, and ID.
            X_id_col (str): Column name for X IDs.
            y_id_col (str): Column name for y IDs.
        Returns:
            DataFrame: Merged and ordered DataFrame.
        """
        # Add a label column to distinguish X and y
        X_df["type"] = "X"
        y_df["type"] = "y"
        
        # Rename ID columns to align for merging
        X_df = X_df.rename(columns={X_id_col: "id"})
        y_df = y_df.rename(columns={y_id_col: "id"})

        # Concatenate X and y DataFrames
        merged_df = pd.concat([X_df, y_df], ignore_index=True)
        
        # Order by id and date
        merged_df = merged_df.sort_values(by=["id", "date"]).reset_index(drop=True)
        
        return merged_df,X_df,y_df
    merged_df,X_df,y_df = merge_and_order_dataframes(X_df, y_df)

    return X, y, dates,X_df,y_df,merged_df

In [ ]:
def ts_dictionary(full_data, X_df, y_df):
    value_to_predict = 'value'
    
    def create_timeseries_dict(grouped_data):
        series_dict = {}
        for encounter_id, group in grouped_data:
            group['date'] = pd.to_datetime(group['date'])  # Ensure 'date' is datetime
            group = group.sort_values(by='date')
            group = group.set_index('date')  # Set 'date' as index
            ts_series = TimeSeries.from_dataframe(group, value_cols=value_to_predict, freq='D')
            series_dict[encounter_id] = ts_series
        return series_dict

    grouped_train = X_df.groupby('id')
    grouped_test = y_df.groupby('id')
    grouped_validation = full_data.groupby('id')

    train_series_dict = create_timeseries_dict(grouped_train)
    test_series_dict = create_timeseries_dict(grouped_test)
    valid_series_dict = create_timeseries_dict(grouped_validation)

    return valid_series_dict, test_series_dict, train_series_dict



In [ ]:
def prepare_train_validation_split(train_series_dict, valid_series_dict, input_chunk_length, output_chunk_length):
    """
    Adjusts train_series_dict to ensure no overlap with targets in valid_series_dict.
    Ensures strict separation to prevent information leakage.
    
    Parameters:
    - train_series_dict: Dictionary of training series (time series without targets).
    - valid_series_dict: Dictionary of validation series (includes prediction targets).
    - input_chunk_length: Length of the input window for the model.
    - output_chunk_length: Length of the prediction horizon.
    
    Returns:
    - adjusted_train_series_dict: Training series dictionary without overlapping target data.
    - valid_series_dict: Validation series dictionary remains unchanged.
    """
    # Initialize the adjusted training series dictionary
    adjusted_train_series_dict = {}
    for key, train_series in train_series_dict.items():
        #print('TRAIN:', key)
        # Check if the key exists in both dictionaries
        if key not in valid_series_dict:
            continue

        # Extract the corresponding validation series
        valid_series = valid_series_dict[key]

        # Identify the last index of the training data
        train_end_idx = len(train_series) - output_chunk_length  # Leave gap for prediction horizon
        #print("train_end_idx", train_end_idx)
        # Truncate training series to avoid overlap with validation targets
        adjusted_train_series = train_series[:train_end_idx]
        adjusted_train_series_dict[key] = adjusted_train_series
    #print('adjusted_train_series_dict:', train_series_dict)
    return train_series_dict, valid_series_dict

    #return train_series_dict, valid_series_dict

In [ ]:
def custom_time_series_split(train_series_dict, valid_series_dict, n_splits):
    """
    Custom time-based splitting function for dictionaries of time series.
    
    Parameters:
    - train_series_dict: Dictionary of training time series.
    - valid_series_dict: Dictionary of validation time series.
    - n_splits: Number of splits for cross-validation.
    
    Returns:
    - A list of (train_keys, valid_keys) pairs for each fold.
    """
    keys = list(train_series_dict.keys())
    #print(keys)
    kf = KFold(n_splits=n_splits, shuffle=False)

    splits = []
    for train_idx, valid_idx in kf.split(keys):
        train_keys = [keys[i] for i in train_idx]
        valid_keys = [keys[i] for i in valid_idx]
        splits.append((train_keys, valid_keys))
    
    return splits

In [ ]:
import pandas as pd
import numpy as np

def resample_ts(filtered_data, resample_rate, prediction_length=None):
    """
    Resample and align time series while preventing temporal information leakage.

    Parameters
    ----------
    filtered_data : pd.DataFrame
        Raw data containing columns:
        ['encounter_id', 'recorded_time', 'code_coding_0_display', 'value']
    resample_rate : str
        Resampling frequency, e.g., '8H' or 'D'.
    prediction_length : int or None
        Optional horizon (in resampling steps) defining the forecast window.
        If provided, data will be truncated up to the forecast origin, ensuring
        no information beyond that point is used for model input.

    Returns
    -------
    pd.DataFrame
        Resampled, leakage-safe time series aligned across variables.
    """

    # --- Step 1: Separate target variable and others ---
    body_temp_df = filtered_data[filtered_data['code_coding_0_display'] == 'Body temperature']
    other_values = filtered_data[filtered_data['code_coding_0_display'] != 'Body temperature']

    # --- Step 2: Define resampling function per encounter ---
    def resample_group(group, freq):
        group = group.set_index('recorded_time').sort_index()
        group = group[~group.index.duplicated(keep='first')]
        aggregated = group.resample(freq).max()  # aggregation is temporally safe
        aggregated['encounter_id'] = group['encounter_id'].iloc[0]
        return aggregated

    # --- Step 3: Apply resampling within each encounter ---
    resampled_df = (
        body_temp_df
        .groupby('encounter_id', group_keys=False)
        .apply(lambda group: resample_group(group, resample_rate))
        .reset_index()
        .rename(columns={'index': 'recorded_time'})
    )

    # --- Step 4: Prepare additional variables for merging ---
    pivoted_other_values = (
        other_values
        .pivot_table(
            index=["encounter_id", "recorded_time"],
            columns="code_coding_0_display",
            values="value",
            aggfunc="max"
        )
        .reset_index()
    )

    # --- Step 5: Temporal alignment using backward-looking merge ---
    # Ensures only past (not future) information is accessible at any timestamp.
    merged_df = pd.merge_asof(
        resampled_df.sort_values("recorded_time"),
        pivoted_other_values.sort_values("recorded_time"),
        on="recorded_time",
        by="encounter_id",
        direction="backward"  # restricts merge to past values only
    )

    merged_df.columns.name = None

    # --- Step 6: Forward-fill missing values (no backward-fill) ---
    # Forward-fill stops at the last valid observed value within each encounter.
    merged_df = (
        merged_df
        .groupby('encounter_id', group_keys=False)
        .apply(lambda g: g.ffill(limit=None))  # forward-fill past values only
        .reset_index(drop=True)
    )

    # --- Step 7: (Optional) Truncate up to the forecast origin ---
    # When a prediction horizon is defined, exclude data beyond that cutoff
    # to simulate realistic forecasting windows.
    if prediction_length is not None:
        truncated = []
        for _, group in merged_df.groupby('encounter_id'):
            if len(group) > prediction_length:
                cutoff_idx = len(group) - prediction_length
                truncated.append(group.iloc[:cutoff_idx])
            else:
                truncated.append(group)
        merged_df = pd.concat(truncated, ignore_index=True)

    # --- Step 8: Fill remaining missing values with global means ---
    missing_columns = merged_df.columns[merged_df.isnull().any()]
    merged_df[missing_columns] = merged_df[missing_columns].fillna(
        merged_df[missing_columns].mean()
    )

    # --- Step 9: Rename columns for clarity ---
    merged_df.rename(columns={
        'Body temperature': 'body_temperature',
        'Heart rate': 'heart_rate',
        'Mean arterial bp': 'mean_arterial_pressure',
        'Oxygen saturation in Arterial blood': 'so2',
        'zBILI': 'bili',
        'zCRP': 'crp',
        'zHB': 'hb',
        'zKREA': 'krea',
        'zLEUA': 'leua'
    }, inplace=True)

    return merged_df


In [ ]:
def bootstrap_metrics_auto(actual_values, predicted_values, n_bootstrap=1000, ci_percentile=95):
    # Initialize arrays to store metrics across bootstraps
    mae_bootstrap = []
    mse_bootstrap = []
    rmse_bootstrap = []
    mape_bootstrap = []
    smape_bootstrap = []

    n_samples = len(actual_values)

    for _ in range(n_bootstrap):
        # Sample with replacement
        resample_indices = np.random.choice(np.arange(n_samples), size=n_samples, replace=True)
        actual_resample = np.array([actual_values[i] for i in resample_indices])
        predicted_resample = np.array([predicted_values[i] for i in resample_indices])

        # Calculate metrics for this bootstrap sample
        mae = np.mean(np.abs(actual_resample - predicted_resample))
        mse = np.mean((actual_resample - predicted_resample) ** 2)
        rmse = np.sqrt(mse)
        mape = np.mean(np.abs((actual_resample - predicted_resample) / actual_resample)) * 100
        smape = np.mean(np.abs(actual_resample - predicted_resample) / (np.abs(actual_resample) + np.abs(predicted_resample))) * 100

        # Append the results
        mae_bootstrap.append(mae)
        mse_bootstrap.append(mse)
        rmse_bootstrap.append(rmse)
        mape_bootstrap.append(mape)
        smape_bootstrap.append(smape)

    # Calculate mean and confidence intervals
    metrics = {
        "mae": (np.mean(mae_bootstrap), np.percentile(mae_bootstrap, (100 - ci_percentile) / 2), np.percentile(mae_bootstrap, 100 - (100 - ci_percentile) / 2)),
        "mse": (np.mean(mse_bootstrap), np.percentile(mse_bootstrap, (100 - ci_percentile) / 2), np.percentile(mse_bootstrap, 100 - (100 - ci_percentile) / 2)),
        "rmse": (np.mean(rmse_bootstrap), np.percentile(rmse_bootstrap, (100 - ci_percentile) / 2), np.percentile(rmse_bootstrap, 100 - (100 - ci_percentile) / 2)),
        "mape": (np.mean(mape_bootstrap), np.percentile(mape_bootstrap, (100 - ci_percentile) / 2), np.percentile(mape_bootstrap, 100 - (100 - ci_percentile) / 2)),
        "smape": (np.mean(smape_bootstrap), np.percentile(smape_bootstrap, (100 - ci_percentile) / 2), np.percentile(smape_bootstrap, 100 - (100 - ci_percentile) / 2)),
    }

    return metrics

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
from autogluon.timeseries import TimeSeriesDataFrame

def autogluon_data_ci_roc(model_name, output_dir_forecasting, output_dir_classification, forecasting_filename, classification_filename, resampled_df, predictor, scaler, prediction_length, value_to_predict, n_bootstrap=1000):
    resampled_df = resampled_df.rename(columns={'id': 'encounter_id', 'date': 'recorded_time'})
    # Split the data into train and test sets
    def split_train_test(group):
        test_rows = group.nlargest(prediction_length, 'recorded_time')
        train_rows = group.drop(test_rows.index)
        return train_rows, test_rows

    train_list = []
    test_list = []

    # Group by encounter_id and split each group
    for name, group in resampled_df.groupby('encounter_id'):
        train_rows, test_rows = split_train_test(group)
        train_list.append(train_rows)
        test_list.append(test_rows)

    # Concatenate the lists into DataFrames
    train_data = pd.concat(train_list).reset_index(drop=True)
    test_data = pd.concat(test_list).reset_index(drop=True)
    train_data = train_data.sort_values(by=['encounter_id', 'recorded_time']).reset_index(drop=True)
    test_data = test_data.sort_values(by=['encounter_id', 'recorded_time']).reset_index(drop=True)

    train_data_ = train_data[['encounter_id', 'recorded_time', value_to_predict]]
    train_data_['recorded_time'] = train_data_['recorded_time'].dt.tz_localize(None)
    test_data_ = test_data[['encounter_id', 'recorded_time', value_to_predict]]
    test_data_['recorded_time'] = test_data_['recorded_time'].dt.tz_localize(None)

    train_data = TimeSeriesDataFrame.from_data_frame(
        train_data_,
        id_column="encounter_id",
        timestamp_column="recorded_time"
    )

    test_data = TimeSeriesDataFrame.from_data_frame(
        test_data_,
        id_column="encounter_id",
        timestamp_column="recorded_time"
    )

    # Generate predictions using AutoGluon predictor
    predictions = predictor.predict(train_data, model=model_name)
    predictions_ = predictions[['mean', '0.05', '0.95']]

    # Join actual and predicted data
    merged_data = test_data.join(predictions_, how="inner")

    actuals = merged_data[value_to_predict].values.reshape(-1, 1)
    predictions = merged_data['mean'].values.reshape(-1, 1)

    # Rescale the actual and predicted values back to the original scale
    def rescale_predictions(scaled_values, scaler):
        if len(scaled_values.shape) == 1:
            scaled_values = scaled_values.reshape(-1, 1)
        return scaler.inverse_transform(scaled_values).flatten()

    actuals = rescale_predictions(actuals, scaler)
    predictions = rescale_predictions(predictions, scaler)

    # Bootstrapping to compute confidence intervals for regression metrics
    metrics_forecasting = bootstrap_metrics_auto(actuals, predictions, n_bootstrap=n_bootstrap)

    # Prepare data for forecasting metrics CSV
    forecasting_metrics_data = []
    for metric, values in metrics_forecasting.items():
        forecasting_metrics_data.append({
            "Metric": metric.upper(),
            "Point Estimate": values[0],
            "95% CI Lower": values[1],
            "95% CI Upper": values[2]
        })

    # Create DataFrame for forecasting metrics
    forecasting_metrics_df = pd.DataFrame(forecasting_metrics_data)

    # Save forecasting metrics
    os.makedirs(output_dir_forecasting, exist_ok=True)
    forecasting_csv_path = os.path.join(output_dir_forecasting, forecasting_filename)
    forecasting_metrics_df.to_csv(forecasting_csv_path, index=False)
    print(f"Forecasting metrics saved to {forecasting_csv_path}")

    # Clinical outcome estimates for classification
    merged_data['actual'] = actuals.flatten()
    merged_data['predictions'] = predictions.flatten()
    fever_flags_actual = merged_data.groupby(level=0)['actual'].transform(lambda x: (x >= 38).any())
    fever_flags_predicted = merged_data.groupby(level=0)['predictions'].transform(lambda x: (x >= 38).any())
    merged_data['actual_fever'] = fever_flags_actual
    merged_data['predictions_fever'] = fever_flags_predicted
    merged_data = merged_data.reset_index(level=0)
    unique_encounters = merged_data.drop_duplicates(subset=['item_id'])

    # Confusion matrix computation
    confusion_conditions = [
        (unique_encounters['actual_fever'] == True) & (unique_encounters['predictions_fever'] == True),  # TP
        (unique_encounters['actual_fever'] == True) & (unique_encounters['predictions_fever'] == False), # FN
        (unique_encounters['actual_fever'] == False) & (unique_encounters['predictions_fever'] == True), # FP
        (unique_encounters['actual_fever'] == False) & (unique_encounters['predictions_fever'] == False) # TN
    ]
    confusion_choices = ['TP', 'FN', 'FP', 'TN']
    unique_encounters['confusion'] = np.select(confusion_conditions, confusion_choices)

    confusion_matrix_bootstrap = {'TP': [], 'FN': [], 'FP': [], 'TN': []}
    metrics_bootstrap = {'sensitivity': [], 'specificity': [], 'ppv': [], 'npv': [], 'accuracy': [], 'f1': [], 'mcc': []}
    auc_scores = []  # For storing AUC scores

    # Bootstrapping for confusion matrix, classification metrics, and AUC
    for _ in range(n_bootstrap):
        bootstrap_sample = unique_encounters.sample(frac=1, replace=True)
        tp = ((bootstrap_sample['actual_fever'] == True) & (bootstrap_sample['predictions_fever'] == True)).sum()
        fn = ((bootstrap_sample['actual_fever'] == True) & (bootstrap_sample['predictions_fever'] == False)).sum()
        fp = ((bootstrap_sample['actual_fever'] == False) & (bootstrap_sample['predictions_fever'] == True)).sum()
        tn = ((bootstrap_sample['actual_fever'] == False) & (bootstrap_sample['predictions_fever'] == False)).sum()

        confusion_matrix_bootstrap['TP'].append(tp)
        confusion_matrix_bootstrap['FN'].append(fn)
        confusion_matrix_bootstrap['FP'].append(fp)
        confusion_matrix_bootstrap['TN'].append(tn)

        metrics_bootstrap['sensitivity'].append(tp / (tp + fn) if (tp + fn) > 0 else 0)
        metrics_bootstrap['specificity'].append(tn / (tn + fp) if (tn + fp) > 0 else 0)
        metrics_bootstrap['ppv'].append(tp / (tp + fp) if (tp + fp) > 0 else 0)
        metrics_bootstrap['npv'].append(tn / (tn + fn) if (tn + fn) > 0 else 0)
        metrics_bootstrap['accuracy'].append((tp + tn) / (tp + tn + fp + fn) if (tp + tn + fp + fn) > 0 else 0)
        metrics_bootstrap['f1'].append((2 * tp) / (2 * tp + fp + fn) if (2 * tp + fp + fn) > 0 else 0)
        metrics_bootstrap['mcc'].append(((tp * tn) - (fp * fn)) / (
            ((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn)) ** 0.5
        ) if ((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn)) > 0 else 0)

        try:
            auc = roc_auc_score(bootstrap_sample['actual_fever'], bootstrap_sample['predictions'])
        except ValueError:
            auc = np.nan  # Handle case where AUC cannot be computed (only one class present)
        auc_scores.append(auc)

    # Confidence intervals for confusion matrix
    confusion_matrix_ci = {}
    for key, values in confusion_matrix_bootstrap.items():
        lower = np.percentile(values, 2.5)
        upper = np.percentile(values, 97.5)
        mean_value = np.mean(values)
        confusion_matrix_ci[key] = {"Mean": mean_value, "CI Lower": lower, "CI Upper": upper}

    # Confidence intervals for classification metrics
    metrics_ci = {}
    for metric, values in metrics_bootstrap.items():
        lower = np.percentile(values, 2.5)
        upper = np.percentile(values, 97.5)
        mean_value = np.mean(values)
        metrics_ci[metric] = {"Mean": mean_value, "CI Lower": lower, "CI Upper": upper}

    # Confidence intervals for AUC
    auc_mean = np.nanmean(auc_scores)
    auc_lower = np.nanpercentile(auc_scores, 2.5)
    auc_upper = np.nanpercentile(auc_scores, 97.5)

    # Prepare data for classification metrics CSV
    classification_metrics_data = []
    for metric, stats in metrics_ci.items():
        classification_metrics_data.append({
            "Metric": metric.capitalize(),
            "Point Estimate": stats["Mean"],
            "95% CI Lower": stats["CI Lower"],
            "95% CI Upper": stats["CI Upper"]
        })

    # Add confusion matrix stats
    for key, stats in confusion_matrix_ci.items():
        classification_metrics_data.append({
            "Metric": f"Avg {key}",
            "Point Estimate": stats["Mean"],
            "95% CI Lower": stats["CI Lower"],
            "95% CI Upper": stats["CI Upper"]
        })

    # Add AUC metric
    classification_metrics_data.append({
        "Metric": "AUC",
        "Point Estimate": auc_mean,
        "95% CI Lower": auc_lower,
        "95% CI Upper": auc_upper
    })

    classification_metrics_df = pd.DataFrame(classification_metrics_data)

    # Save classification metrics
    os.makedirs(output_dir_classification, exist_ok=True)
    classification_csv_path = os.path.join(output_dir_classification, classification_filename)
    classification_metrics_df.to_csv(classification_csv_path, index=False)
    print(f"Classification metrics saved to {classification_csv_path}")

    # Plot ROC curve and save it
    def plot_roc_curve(actuals, predictions, output_path):
        fpr, tpr, _ = roc_curve(actuals, predictions)
        auc = roc_auc_score(actuals, predictions)

        plt.figure(figsize=(8, 6))
        plt.plot(fpr, tpr, label=f'ROC Curve (AUC = {auc:.2f})')
        plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Receiver Operating Characteristic (ROC) Curve')
        plt.legend(loc='lower right')
        plt.grid(True)
        plt.savefig(output_path)
        plt.close()
        print(f"ROC curve saved to {output_path}")

    # Save the ROC curve plot
    roc_curve_path = os.path.join(output_dir_classification, 'roc_curve.png')
    plot_roc_curve(unique_encounters['actual_fever'], unique_encounters['predictions'], roc_curve_path)

    return predictions, actuals, train_data, test_data, merged_data,unique_encounters['actual_fever'],unique_encounters['predictions']


In [ ]:
def timellm_data_ci_roc_(model_name, output_dir_forecasting, output_dir_classification, forecasting_filename, classification_filename, resampled_df,nf, scaler, prediction_length, value_to_predict, n_bootstrap=1000):
    
    full_data_test = resampled_df.groupby("id").tail(prediction_length)
    full_data_train = resampled_df.drop(full_data_test.index)
    full_data_train_ = full_data_train.rename(columns={'date': 'ds', 'id': 'unique_id','value':'y'})
    full_data_test_ = full_data_test.rename(columns={'date': 'ds', 'id': 'unique_id','value':'y'})
    full_data_train_ = full_data_train_.drop(columns=['type'])
    full_data_test_ = full_data_test_.drop(columns=['type'])
    forecasts = nf.predict(full_data_train_)
    def rescale_to_original(scaled_df, scaler, columns_to_rescale):
        scaled_df[columns_to_rescale] = scaler.inverse_transform(scaled_df[columns_to_rescale])
        return scaled_df
    columns_to_rescale_actual = ["y"]
    columns_to_rescale_predict = ["TimeLLM"]
    actuals = rescale_to_original(full_data_test_, scaler,columns_to_rescale_actual)
    predictions = rescale_to_original(forecasts, scaler,columns_to_rescale_predict)

    def bootstrap_timellm_metrics_length(forecasts_df, actual_values_df, prediction_length, n_bootstrap, ci_percentile=95):
        mae_bootstrap = []
        mse_bootstrap = []
        rmse_bootstrap = []
        smape_bootstrap = []
        mape_bootstrap = []

        # Merge forecasts and actual values on 'unique_id' and 'ds'
        merged_df = pd.merge(forecasts_df.reset_index(), actual_values_df, on=['unique_id', 'ds'])

        unique_ids = merged_df['unique_id'].unique()
        
        for _ in range(n_bootstrap):
            # Sample with replacement
            sampled_ids = np.random.choice(unique_ids, size=len(unique_ids), replace=True)
            sampled_data = merged_df[merged_df['unique_id'].isin(sampled_ids)]

            total_mae = 0
            total_mse = 0
            total_smape = 0
            total_mape = 0

            for unique_id in sampled_ids:
                # Select the forecast and true values for this unique_id
                data = sampled_data[sampled_data['unique_id'] == unique_id].sort_values(by='ds')
                
                forecast_values = data['TimeLLM'].values[-prediction_length:]
                true_values = data['y'].values[-prediction_length:]

                # Compute errors
                absolute_errors = np.abs(forecast_values - true_values)
                
                # MAE
                mae = absolute_errors.mean()
                total_mae += mae

                # MSE
                mse = (absolute_errors ** 2).mean()
                total_mse += mse

                # SMAPE
                denominator = (np.abs(true_values) + np.abs(forecast_values)) / 2
                smape = (absolute_errors / denominator).mean() * 100
                total_smape += smape

                # MAPE
                mape = (np.abs((true_values - forecast_values) / true_values).mean()) * 100
                total_mape += mape

            # Compute overall metrics for this bootstrap sample
            mae_bootstrap.append(total_mae / len(sampled_ids))
            mse_bootstrap.append(total_mse / len(sampled_ids))
            rmse_bootstrap.append(np.sqrt(total_mse / len(sampled_ids)))
            smape_bootstrap.append(total_smape / len(sampled_ids))
            mape_bootstrap.append(total_mape / len(sampled_ids))

        # Calculate mean and confidence intervals
        metrics = {
            "mae": (
                np.mean(mae_bootstrap),
                np.percentile(mae_bootstrap, (100 - ci_percentile) / 2),
                np.percentile(mae_bootstrap, 100 - (100 - ci_percentile) / 2),
            ),
            "mse": (
                np.mean(mse_bootstrap),
                np.percentile(mse_bootstrap, (100 - ci_percentile) / 2),
                np.percentile(mse_bootstrap, 100 - (100 - ci_percentile) / 2),
            ),
            "rmse": (
                np.mean(rmse_bootstrap),
                np.percentile(rmse_bootstrap, (100 - ci_percentile) / 2),
                np.percentile(rmse_bootstrap, 100 - (100 - ci_percentile) / 2),
            ),
            "smape": (
                np.mean(smape_bootstrap),
                np.percentile(smape_bootstrap, (100 - ci_percentile) / 2),
                np.percentile(smape_bootstrap, 100 - (100 - ci_percentile) / 2),
            ),
            "mape": (
                np.mean(mape_bootstrap),
                np.percentile(mape_bootstrap, (100 - ci_percentile) / 2),
                np.percentile(mape_bootstrap, 100 - (100 - ci_percentile) / 2),
            ),
        }

        return metrics, merged_df

    # Example usage
    metrics, merged_data = bootstrap_timellm_metrics_length(
        forecasts_df=predictions,
        actual_values_df=actuals,
        prediction_length=prediction_length,  # Use the desired prediction length
        n_bootstrap=1000,     # Number of bootstrap samples
        ci_percentile=95      # Confidence interval
    )




    print(merged_data.info())


    # Create DataFrame for forecasting metrics
    forecasting_metrics_df = pd.DataFrame(metrics)

    # Save forecasting metrics
    os.makedirs(output_dir_forecasting, exist_ok=True)
    forecasting_csv_path = os.path.join(output_dir_forecasting, forecasting_filename)
    forecasting_metrics_df.to_csv(forecasting_csv_path, index=False)
    print(f"Forecasting metrics saved to {forecasting_csv_path}")
    merged_data = merged_data.rename(columns={'y': 'actual', 'TimeLLM': 'predictions'})

    #merged_data['actual'] = actuals.flatten()
    #merged_data['predictions'] = predictions.flatten()
    fever_flags_actual = merged_data.groupby(level=0)['actual'].transform(lambda x: (x >= 38).any())
    fever_flags_predicted = merged_data.groupby(level=0)['predictions'].transform(lambda x: (x >= 38).any())
    merged_data['actual_fever'] = fever_flags_actual
    merged_data['predictions_fever'] = fever_flags_predicted
    merged_data = merged_data.reset_index(level=0)
    unique_encounters = merged_data.drop_duplicates(subset=['unique_id'])
    print(unique_encounters.info())
    # Confusion matrix computation
    confusion_conditions = [
        (unique_encounters['actual_fever'] == True) & (unique_encounters['predictions_fever'] == True),  # TP
        (unique_encounters['actual_fever'] == True) & (unique_encounters['predictions_fever'] == False), # FN
        (unique_encounters['actual_fever'] == False) & (unique_encounters['predictions_fever'] == True), # FP
        (unique_encounters['actual_fever'] == False) & (unique_encounters['predictions_fever'] == False) # TN
    ]
    confusion_choices = ['TP', 'FN', 'FP', 'TN']
    unique_encounters['confusion'] = np.select(confusion_conditions, confusion_choices)

    confusion_matrix_bootstrap = {'TP': [], 'FN': [], 'FP': [], 'TN': []}
    metrics_bootstrap = {'sensitivity': [], 'specificity': [], 'ppv': [], 'npv': [], 'accuracy': [], 'f1': [], 'mcc': []}
    auc_scores = []  # For storing AUC scores

    # Bootstrapping for confusion matrix, classification metrics, and AUC
    for _ in range(n_bootstrap):
        bootstrap_sample = unique_encounters.sample(frac=1, replace=True)
        tp = ((bootstrap_sample['actual_fever'] == True) & (bootstrap_sample['predictions_fever'] == True)).sum()
        fn = ((bootstrap_sample['actual_fever'] == True) & (bootstrap_sample['predictions_fever'] == False)).sum()
        fp = ((bootstrap_sample['actual_fever'] == False) & (bootstrap_sample['predictions_fever'] == True)).sum()
        tn = ((bootstrap_sample['actual_fever'] == False) & (bootstrap_sample['predictions_fever'] == False)).sum()

        confusion_matrix_bootstrap['TP'].append(tp)
        confusion_matrix_bootstrap['FN'].append(fn)
        confusion_matrix_bootstrap['FP'].append(fp)
        confusion_matrix_bootstrap['TN'].append(tn)

        metrics_bootstrap['sensitivity'].append(tp / (tp + fn) if (tp + fn) > 0 else 0)
        metrics_bootstrap['specificity'].append(tn / (tn + fp) if (tn + fp) > 0 else 0)
        metrics_bootstrap['ppv'].append(tp / (tp + fp) if (tp + fp) > 0 else 0)
        metrics_bootstrap['npv'].append(tn / (tn + fn) if (tn + fn) > 0 else 0)
        metrics_bootstrap['accuracy'].append((tp + tn) / (tp + tn + fp + fn) if (tp + tn + fp + fn) > 0 else 0)
        metrics_bootstrap['f1'].append((2 * tp) / (2 * tp + fp + fn) if (2 * tp + fp + fn) > 0 else 0)
        metrics_bootstrap['mcc'].append(((tp * tn) - (fp * fn)) / (
            ((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn)) ** 0.5
        ) if ((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn)) > 0 else 0)

        try:
            auc = roc_auc_score(bootstrap_sample['actual_fever'], bootstrap_sample['predictions'])
        except ValueError:
            auc = np.nan  # Handle case where AUC cannot be computed (only one class present)
        auc_scores.append(auc)

    # Confidence intervals for confusion matrix
    confusion_matrix_ci = {}
    for key, values in confusion_matrix_bootstrap.items():
        lower = np.percentile(values, 2.5)
        upper = np.percentile(values, 97.5)
        mean_value = np.mean(values)
        confusion_matrix_ci[key] = {"Mean": mean_value, "CI Lower": lower, "CI Upper": upper}

    # Confidence intervals for classification metrics
    metrics_ci = {}
    for metric, values in metrics_bootstrap.items():
        lower = np.percentile(values, 2.5)
        upper = np.percentile(values, 97.5)
        mean_value = np.mean(values)
        metrics_ci[metric] = {"Mean": mean_value, "CI Lower": lower, "CI Upper": upper}

    # Confidence intervals for AUC
    auc_mean = np.nanmean(auc_scores)
    auc_lower = np.nanpercentile(auc_scores, 2.5)
    auc_upper = np.nanpercentile(auc_scores, 97.5)

    # Prepare data for classification metrics CSV
    classification_metrics_data = []
    for metric, stats in metrics_ci.items():
        classification_metrics_data.append({
            "Metric": metric.capitalize(),
            "Point Estimate": stats["Mean"],
            "95% CI Lower": stats["CI Lower"],
            "95% CI Upper": stats["CI Upper"]
        })

    # Add confusion matrix stats
    for key, stats in confusion_matrix_ci.items():
        classification_metrics_data.append({
            "Metric": f"Avg {key}",
            "Point Estimate": stats["Mean"],
            "95% CI Lower": stats["CI Lower"],
            "95% CI Upper": stats["CI Upper"]
        })

    # Add AUC metric
    classification_metrics_data.append({
        "Metric": "AUC",
        "Point Estimate": auc_mean,
        "95% CI Lower": auc_lower,
        "95% CI Upper": auc_upper
    })

    classification_metrics_df = pd.DataFrame(classification_metrics_data)

    # Save classification metrics
    os.makedirs(output_dir_classification, exist_ok=True)
    classification_csv_path = os.path.join(output_dir_classification, classification_filename)
    classification_metrics_df.to_csv(classification_csv_path, index=False)
    print(f"Classification metrics saved to {classification_csv_path}")

    # Plot ROC curve and save it
    def plot_roc_curve(actuals, predictions, output_path):
        fpr, tpr, _ = roc_curve(actuals, predictions)
        auc = roc_auc_score(actuals, predictions)

        plt.figure(figsize=(8, 6))
        plt.plot(fpr, tpr, label=f'ROC Curve (AUC = {auc:.2f})')
        plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Receiver Operating Characteristic (ROC) Curve')
        plt.legend(loc='lower right')
        plt.grid(True)
        plt.savefig(output_path)
        plt.close()
        print(f"ROC curve saved to {output_path}")

    # Save the ROC curve plot
    roc_curve_path = os.path.join(output_dir_classification, 'roc_curve.png')
    plot_roc_curve(unique_encounters['actual_fever'], unique_encounters['predictions'], roc_curve_path)

    return predictions, actuals, merged_data,unique_encounters['actual_fever'],unique_encounters['predictions']


